Code for generating results for the confounder-mediator graph (Figure 2(c) and Figure 3(c)).

In [ ]:
import numpy
import sympy
import pandas
import numpy as np
import pandas as pd
import sympy as sp
import datetime
import copy
import attr
import time
import logging
import itertools
import pickle
import sys
import os
import functools

import matplotlib.pyplot as plt
from matplotlib import rcParams
import collections

from scipy.optimize import minimize
from scipy import interpolate
import warnings
from IPython.display import clear_output

HOME_DIR = ""
warnings.simplefilter(action='ignore', category=UserWarning)

In [ ]:
class ModelParams:

  def __init__(self, beta, a, b, d, w, x, m, y):
    self.beta = beta
    self.a = a
    self.b = b
    self.d = d
    self.w = w
    self.x = x
    self.m = m
    self.y = y
  
  def get_true_param(self):
    return self.beta
  
  def __str__(self):
    return "beta=%f, a=%f, b=%f, d=%f, w=%f, x=%f, m=%f, y=%f" % (self.beta, self.a, self.b, self.d, self.w, self.x, self.m, self.y)

truth = ModelParams(beta=-0.32439283544223846, a=0.3335611397940348, b=-0.34589828079316454, d=0.44690979199821323, w=1, x=1, m=1, y=1)

In [ ]:
def generate_data_samples(num_samples, model, k=0.5):
  np = numpy
  pd = pandas

  u_w = np.random.normal(scale=np.sqrt(model.w), size=(num_samples,))
  u_x = np.random.normal(scale=np.sqrt(model.x), size=(num_samples,))
  u_m = np.random.normal(scale=np.sqrt(model.m), size=(num_samples,))
  u_y = np.random.normal(scale=np.sqrt(model.y), size=(num_samples,))

  def get_W(u):
    return u
  
  def get_X(W, u):
    return model.d * W + u

  def get_M(X, u):
    return (model.beta / model.a) * X + u
  
  def get_Y(M, W, u):
    return model.a * M + model.b * W + u
  
  W = get_W(u_w)
  X = get_X(W, u_x)
  M = get_M(X, u_m)
  Y = get_Y(M, W, u_y)
  SEL = np.zeros_like(Y, np.int32)
  SEL[:int(SEL.shape[0] * k)] = 1

  df = pd.DataFrame({
      "W": W,
      "X": X,
      "M": M,
      "Y": Y,
      "SEL": SEL
  })
  return df

In [ ]:
num_samples = 1000
df = generate_data_samples(num_samples, truth)
df.head()

,W,X,M,Y,SEL
0,0.356817,-1.173050,1.481314,1.193225,1
1,0.548864,0.310346,-0.172684,-0.440570,1
2,0.686575,-0.418906,-0.327433,-0.657431,1
3,-1.864017,-0.253947,-1.388078,-0.277440,1
4,-0.440858,-1.019033,0.990077,1.321674,1


In [ ]:
def get_fraction_cost(cost, k):
  return 1 + (cost - 1) * k

In [ ]:
class GMMEqs:

  def __init__(self):
    self.moment_eqs, self.jacobian_eqs, self.moment_reweighting = self._get_equations()
  
  def get(self):
    return self.moment_eqs, self.jacobian_eqs, self.moment_reweighting
  
  def _get_equations(self):
    sp = sympy

    df_symbols = sp.symbols('s1, s0, S, W, X, M, Y')
    s1, s0, S, W, X, M, Y = df_symbols
    params = sp.symbols("t, a, b, d, w, x")
    t, a, b, d, w, x = params

    m_wts = [s1, s1, s0, s0, s0, s1, s1, s1, 1]
    moment_reweighting = sp.zeros(len(m_wts), len(m_wts))
    for i, mw1 in enumerate(m_wts):
      for j, mw2 in enumerate(m_wts):
        if mw1 == 1 or mw2 == 1:
          moment_reweighting[i, j] = mw1 * mw2
        elif mw1 == mw2:
          moment_reweighting[i, j] = mw1

    m1 = m_wts[0] * S * X * (Y - b*W - t * X)
    m2 = m_wts[1] * S * W * (Y - b*W - t * X)
    m3 = m_wts[2] * (1 - S) * X * (M - (t/a) * X)
    m4 = m_wts[3] * (1 - S) * M * (Y - a*M - (b*d*w)/(d**2 * w + x) * X)
    m5 = m_wts[4] * (1 - S) * X * (Y - a*M - (b*d*w)/(d**2 * w + x) * X)
    m6 = m_wts[5] * S * (W**2 - w)
    m7 = m_wts[6] * S * (X**2 - d**2 * w - x)
    m8 = m_wts[7] * S * W * (X - d*W)
    m9 = m_wts[8] * (X**2 - (d**2 * w + x))

    all_symbols = df_symbols + params
    moments = [m1, m2, m3, m4, m5, m6, m7, m8, m9]
    jacobian = []
    for mom in moments:
      jac_row = []
      for p in params:
        eq = sp.simplify(sp.diff(mom, p))
        jac_row.append(sp.lambdify(all_symbols, eq, "numpy"))
      jacobian.append(jac_row)
    
    moments = [sp.lambdify(all_symbols, eq, "numpy") for eq in moments]
    moment_reweighting = sp.lambdify((s1, s0), moment_reweighting, "numpy")
    return moments, jacobian, moment_reweighting

class GMM:

  def __init__(self, df, gmm_eqs):
    np = numpy
    
    self.df_dict = {k: df[k].values for k in df.columns}
    self.df = self.df_dict
    self.moment_eqs, self.jacobian_eqs, self.moment_reweighting = gmm_eqs.get()
    self.num_samples = len(df)
    self.num_moments = len(self.moment_eqs)
    self.momconds_arr = np.zeros((self.num_moments, self.num_samples))
  
  def momconds(self, params):
    """Returns the emprical moment vector (equivalent of \hat{g}_n(\theta))."""
    np = numpy
    n = self.num_samples
    return np.sum(self._momconds_arr(params), axis=-1) / n
  
  def _compute_moment_covariance(self, params):
    """Estimate the optimal weight matrix using the emprical moments."""
    n = self.num_samples
    moms = self._momconds_arr(params)
    moment_covariance = (moms @ moms.T) / n
    return moment_covariance
  
  def _get_objective_fn(self, weight_matrix_inv):
    """Returns the GMM objective function."""
    np = numpy
    
    def objective(params):
      moms = self.momconds(params)
      w_inv_mom = np.linalg.solve(weight_matrix_inv, moms)
      obj = moms.T @ w_inv_mom

      return obj
    
    return objective
  
  def _momconds_arr(self, params):
    np = numpy

    p = params
    df = self.df_dict
    n = self.num_samples

    for i, eq in enumerate(self.moment_eqs):
      self.momconds_arr[i, :] = eq(S=df["SEL"], W=df["W"], X=df["X"],
                                   M=df["M"], Y=df["Y"],
                                   s0=1, s1=1,
                                   t=p[0], a=p[1], b=p[2], d=p[3], w=p[4], x=p[5])
    
    return self.momconds_arr

  def _get_asymptotic_variance_matrix(self, k, moment_covariance, params):
    np = numpy

    current_k = np.mean(self.df["SEL"])

    p = params
    df = self.df
    n = self.num_samples

    jacobian = np.zeros((self.num_moments, len(p)))

    for i, jac_row in enumerate(self.jacobian_eqs):
      for j, eq in enumerate(jac_row):
        jacobian[i, j] = np.sum(eq(S=df["SEL"], W=df["W"], X=df["X"],
                                   M=df["M"], Y=df["Y"],
                                   s1=k/current_k, s0=(1-k)/(1-current_k),
                                   t=p[0], a=p[1], b=p[2], d=p[3], w=p[4], x=p[5])) / n

    moment_covariance_reweight = self.moment_reweighting(s1=k/current_k, s0=(1-k)/(1-current_k))
    
    moment_covariance = moment_covariance * moment_covariance_reweight

    mom_cov_inv_jac = np.linalg.solve(moment_covariance, jacobian)
    return np.linalg.inv(jacobian.T @ mom_cov_inv_jac)
  
  def _get_asymptotic_variance(self, k, moment_covariance, params):
    return self._get_asymptotic_variance_matrix(k, moment_covariance, params)[0][0]
  
  def _optimize_find_parameters(self, weight_matrix_inv, initial_guess=None):
    np = numpy

    initial_guess = np.array([truth.beta, truth.a, truth.b, truth.d, truth.w, truth.x]) * 1 if initial_guess is None else initial_guess
    res = minimize(self._get_objective_fn(weight_matrix_inv), initial_guess,
                   bounds=[
                           (-np.inf, np.inf),
                           (-np.inf, np.inf),
                           (-np.inf, np.inf),
                           (-np.inf, np.inf),
                           (0.01, np.inf),
                           (0.01, np.inf),
                   ],
                  )
    return res.x
  
  def find_parameters(self, num_iters=2, weight_matrix_reg=None):
    np = numpy

    weight_matrix_inv = np.eye(self.num_moments)
    for i in range(num_iters):
      params = self._optimize_find_parameters(weight_matrix_inv, initial_guess=params if i > 0 else None)
      weight_matrix_inv = self._compute_moment_covariance(params)
      
      if weight_matrix_reg is not None:
        weight_matrix_inv += weight_matrix_reg * np.eye(weight_matrix_inv.shape[0])
    
    return params, weight_matrix_inv
  
  def find_optimal_k(self, moment_covariance, params, cost=1):
    np = numpy
    
    initial_guess = np.array([0.5])
    lower_bound = 0.05
    upper_bound = 0.95
    objective = lambda x: self._get_asymptotic_variance(x[0], moment_covariance, params) * get_fraction_cost(cost, x)
    res = minimize(objective,
                   initial_guess, bounds=[(lower_bound, upper_bound)])
    
    if res.x[0] == lower_bound:
      return 0
    
    if res.x[0] == upper_bound:
      return 1
      
    return res.x[0]

In [ ]:
gmm = GMM(df, GMMEqs())
params, moment_covariance = gmm.find_parameters(num_iters=2)

print("True params:")
print(truth)
print("Estimated params:")
print(params)

optimal_k = gmm.find_optimal_k(moment_covariance, params)
print("Estimated optimal selection ratio k: %f" % optimal_k)

del gmm, params, moment_covariance, optimal_k

True params:
beta=-0.324393, a=0.333561, b=-0.345898, d=0.446910, w=1.000000, x=1.000000, m=1.000000, y=1.000000
Estimated params:
[-0.29488337  0.31020581 -0.33048556  0.40852891  1.08834502  1.03988199]
Estimated optimal selection ratio k: 0.329303


In [ ]:
class SampleRevealer:

  def __init__(self, budget, cost, df):
    self.initial_budget = budget
    self.budget = budget
    self.cost = cost
    self.buffer_size = len(df)
    self.df = df
    self.counter = 0

  def reset(self):
    self.counter = 0
    self.budget = self.initial_budget
  
  def is_budget_left(self, samples_to_reveal, k):
    return self.budget >= samples_to_reveal * get_fraction_cost(self.cost, k)
  
  def reveal(self, reveal_k, samples_to_reveal):
    if self.counter + samples_to_reveal >= self.buffer_size:
      raise ValueError("no buffer")
    
    if not self.is_budget_left(samples_to_reveal, reveal_k):
      raise ValueError("no budget left")
    
    observe_count = int(reveal_k * samples_to_reveal)
    self.df["SEL"].values[self.counter:self.counter+observe_count] = 1
    self.df["SEL"].values[self.counter+observe_count:self.counter+samples_to_reveal] = 0

    self.counter += samples_to_reveal
    self.budget -= (samples_to_reveal * get_fraction_cost(self.cost, reveal_k))
  
  def reveal_with_budget(self, reveal_k, budget_to_reveal):
    samples_to_reveal = int(budget_to_reveal / get_fraction_cost(self.cost, reveal_k))
    self.reveal(reveal_k, samples_to_reveal)
  
  def get_dataset(self):
    return self.df[:self.counter]

In [ ]:
def batch_fractions_to_sizes(horizon, batch_fractions):
  batch_sizes = []
    
  for i in range(len(batch_fractions) + 1):
    prev_batch_end = 0 if i == 0 else int(horizon * batch_fractions[i - 1])
    curr_batch_end = horizon if i == len(batch_fractions) else int(horizon * batch_fractions[i])

    batch_sizes.append(curr_batch_end - prev_batch_end)
  
  return batch_sizes

In [ ]:
def compute_reveal_k(current_samples, samples_to_reveal, current_k, target_k):
  reveal_k = (target_k * (current_samples + samples_to_reveal) - current_k * current_samples) / (samples_to_reveal)
  return min(1, (max(0, reveal_k)))

def compute_reveal_k_with_budget(current_samples, budget_to_reveal, cost, current_k, target_k):
  np = numpy
  delta = 0.01
  
  best_k = -1
  best_val = 2
  for k in np.arange(0, 1 + delta, delta):
    samples_to_reveal = int(budget_to_reveal / (1 + (cost-1)*k))
    final_k = (current_samples * current_k + samples_to_reveal * k) / (current_samples + samples_to_reveal)

    dist = (final_k - target_k)**2
    if dist < best_val:
      best_val = dist
      best_k = k

  return best_k

In [ ]:
class StrategyRunResult:

  def __init__(self):
    self.budgets_used = []
    self.squared_errors = []
    self.optimal_ks = []
    self.current_ks = []
  
  def append(self, budget_used, squared_error, optimal_k, current_k):
    self.budgets_used.append(budget_used)
    self.squared_errors.append(squared_error)
    self.optimal_ks.append(optimal_k)
    self.current_ks.append(current_k)


In [ ]:
class OracleStrategy:

  def __init__(self, sample_revealer, gmm_equations, optimal_k, cost, budget, batch_fractions):
    self.sample_revealer = sample_revealer
    self.name = 'oracle'

    # assume that batches are budget fraction allocations
    self.batch_sizes = batch_fractions_to_sizes(budget, batch_fractions)
    self.optimal_k = optimal_k
    self.cost = cost
    self.gmm_equations = gmm_equations
  
  def get_current_df_vals(self):
    np = numpy

    dataset = self.sample_revealer.get_dataset()

    len_left_corner = np.sum(dataset["SEL"])
    len_right_corner = len(dataset) - len_left_corner
    total = (len_left_corner + len_right_corner)
    return len_left_corner / total, total

  def can_step(self):
    return self.sample_revealer.is_budget_left()
  
  def get_and_store_params(self):
    dataset = self.sample_revealer.get_dataset()
    self.gmm = GMM(dataset, self.gmm_equations)
    self.params, self.moment_covariance = self.gmm.find_parameters(num_iters=2)
    return self.params

  def get_squared_error(self):
    error = (truth.get_true_param() - self.params[0])**2
    # print(error)
    return error
  
  def execute_run(self, cost=1):
    np = numpy

    result = StrategyRunResult()
    for i, batch_size in enumerate(self.batch_sizes):
      if i == 0:
        current_k, current_samples = 0, 0
      else:
        current_k, current_samples = self.get_current_df_vals()

      reveal_k = compute_reveal_k_with_budget(current_samples, batch_size, self.cost, current_k, self.optimal_k)

      self.sample_revealer.reveal_with_budget(reveal_k=reveal_k, budget_to_reveal=batch_size)
      _ = self.get_and_store_params()

      current_k, _ = self.get_current_df_vals()

      result.append(
          self.sample_revealer.initial_budget - self.sample_revealer.budget,
          self.get_squared_error(), self.optimal_k, current_k,
      )
    
    return result

In [ ]:
class ETCStrategy:

  def __init__(self, sample_revealer, gmm_equations, cost, budget, batch_fractions):
    self.sample_revealer = sample_revealer
    self.name = 'etc'
    # Batch sizes represent fractions of budget allocations.
    # We assume that the first batch size is exploration.
    self.batch_sizes = batch_fractions_to_sizes(budget, batch_fractions)
    self.cost = cost
    self.gmm_equations = gmm_equations
  
  def get_current_df_vals(self):
    np = numpy

    dataset = self.sample_revealer.get_dataset()

    len_left_corner = np.sum(dataset["SEL"])
    len_right_corner = len(dataset) - len_left_corner
    total = (len_left_corner + len_right_corner)
    return len_left_corner / total, total

  def can_step(self):
    return self.sample_revealer.is_budget_left()
  
  def get_and_store_params(self):
    dataset = self.sample_revealer.get_dataset()
    self.gmm = GMM(dataset, self.gmm_equations)
    self.params, self.moment_covariance = self.gmm.find_parameters(num_iters=2)
    return self.params

  def get_squared_error(self):
    error = (truth.get_true_param() - self.params[0])**2
    return error
  
  def execute_run(self, cost=1):
    np = numpy

    result = StrategyRunResult()
    for i, batch_size in enumerate(self.batch_sizes):
      if i == 0:
        reveal_k = 0.5
      else:
        current_k, current_samples = self.get_current_df_vals()
        reveal_k = compute_reveal_k_with_budget(current_samples, batch_size,
                                                self.cost, current_k, self.optimal_k)

      self.sample_revealer.reveal_with_budget(reveal_k=reveal_k, budget_to_reveal=batch_size)
      params = self.get_and_store_params()

      if i == 0:
        self.optimal_k = self.gmm.find_optimal_k(self.moment_covariance, params, cost=cost)

      current_k, _ = self.get_current_df_vals()

      result.append(
          self.sample_revealer.initial_budget - self.sample_revealer.budget,
          self.get_squared_error(), self.optimal_k, current_k,
      )
    
    return result

In [ ]:
class ETGreedyFBStrategy:

  def __init__(self, sample_revealer, gmm_equations, cost, budget, batch_fractions):
    self.sample_revealer = sample_revealer
    self.name = 'etg-fb'
    # We assume that the first batch size is exploration.
    self.batch_sizes = batch_fractions_to_sizes(budget, batch_fractions)
    self.cost = cost
    self.gmm_equations = gmm_equations
    self.weight_matrix_reg = 0.01
  
  def get_current_df_vals(self):
    np = numpy

    dataset = self.sample_revealer.get_dataset()

    len_left_corner = np.sum(dataset["SEL"])
    len_right_corner = len(dataset) - len_left_corner
    total = (len_left_corner + len_right_corner)
    return len_left_corner / total, total

  def can_step(self):
    return self.sample_revealer.is_budget_left()
  
  def get_and_store_params(self, is_last_step):
    dataset = self.sample_revealer.get_dataset()
    self.gmm = GMM(dataset, self.gmm_equations)
    if is_last_step:
      self.params, self.moment_covariance = self.gmm.find_parameters(num_iters=2)
    else:
      self.params, self.moment_covariance = (
          self.gmm.find_parameters(num_iters=2, weight_matrix_reg=self.weight_matrix_reg)
      )
    return self.params

  def get_squared_error(self):
    error = (truth.get_true_param() - self.params[0])**2
    return error
  
  def execute_run(self, cost=1):
    np = numpy

    result = StrategyRunResult()
    for i, batch_size in enumerate(self.batch_sizes):
      if i == 0:
        reveal_k = 0.5
      else:
        current_k, current_samples = self.get_current_df_vals()
        reveal_k = compute_reveal_k_with_budget(current_samples, batch_size, self.cost, current_k, self.optimal_k)

      self.sample_revealer.reveal_with_budget(reveal_k=reveal_k, budget_to_reveal=batch_size)
      is_last_step = (i==len(self.batch_sizes)-1)
      params = self.get_and_store_params(is_last_step)

      self.optimal_k = self.gmm.find_optimal_k(self.moment_covariance, params, cost=cost)

      current_k, _ = self.get_current_df_vals()

      result.append(
          self.sample_revealer.initial_budget - self.sample_revealer.budget,
          self.get_squared_error(), self.optimal_k, current_k,
      )
    
    return result

In [ ]:
class ETGreedyFSStrategy:

  def __init__(self, sample_revealer, gmm_equations, cost, budget, batch_fractions):
    self.sample_revealer = sample_revealer
    self.name = 'etg-fs'
    horizon_min = int(budget / max(get_fraction_cost(cost, 0), get_fraction_cost(cost, 1))) 
    # We assume that the first batch size is exploration.
    self.batch_sizes = batch_fractions_to_sizes(horizon_min, batch_fractions)[:-1]
    self.cost = cost
    self.gmm_equations = gmm_equations
    self.weight_matrix_reg = 0.01
  
  def get_current_df_vals(self):
    np = numpy

    dataset = self.sample_revealer.get_dataset()

    len_left_corner = np.sum(dataset["SEL"])
    len_right_corner = len(dataset) - len_left_corner
    total = (len_left_corner + len_right_corner)
    return len_left_corner / total, total

  def can_step(self):
    return self.sample_revealer.is_budget_left()
  
  def get_and_store_params(self, is_last_step):
    dataset = self.sample_revealer.get_dataset()
    self.gmm = GMM(dataset, self.gmm_equations)
    if is_last_step:
      self.params, self.moment_covariance = self.gmm.find_parameters(num_iters=2)
    else:
      self.params, self.moment_covariance = (
          self.gmm.find_parameters(num_iters=2, weight_matrix_reg=self.weight_matrix_reg)
      )
    return self.params

  def get_squared_error(self):
    error = (truth.get_true_param() - self.params[0])**2
    return error
  
  def execute_run(self, cost=1):
    np = numpy

    result = StrategyRunResult()
    for i, batch_size in enumerate(self.batch_sizes):
      if i == 0:
        reveal_k = 0.5
      else:
        current_k, current_samples = self.get_current_df_vals()
        reveal_k = compute_reveal_k(current_samples, batch_size, current_k, self.optimal_k)

      self.sample_revealer.reveal(reveal_k=reveal_k, samples_to_reveal=batch_size)
      is_last_step = (i==len(self.batch_sizes)-1)
      params = self.get_and_store_params(is_last_step)

      self.optimal_k = self.gmm.find_optimal_k(self.moment_covariance, params, cost=cost)

      current_k, current_samples = self.get_current_df_vals()

      result.append(
          self.sample_revealer.initial_budget - self.sample_revealer.budget,
          self.get_squared_error(), self.optimal_k, current_k,
      )
    
    last_step_reached = False
    batch_size = self.batch_sizes[-1]
    while not last_step_reached:      
      current_k, current_samples = self.get_current_df_vals()
      reveal_k = compute_reveal_k(current_samples, batch_size, current_k, self.optimal_k)

      if self.sample_revealer.is_budget_left(samples_to_reveal=batch_size, k=reveal_k):
        self.sample_revealer.reveal(reveal_k=reveal_k, samples_to_reveal=batch_size)
      else:
        budget_left = self.sample_revealer.budget
        reveal_k = compute_reveal_k_with_budget(current_samples, budget_left, self.cost, current_k, self.optimal_k)
        self.sample_revealer.reveal_with_budget(reveal_k=reveal_k, budget_to_reveal=budget_left)
        last_step_reached = True
      
      params = self.get_and_store_params(last_step_reached)
      self.optimal_k = self.gmm.find_optimal_k(self.moment_covariance, params, cost=cost)

      current_k, current_samples = self.get_current_df_vals()

      result.append(
          self.sample_revealer.initial_budget - self.sample_revealer.budget,
          self.get_squared_error(), self.optimal_k, current_k,
      )
    
    return result

In [ ]:
def execute_strategy_iteration(strategy_name, iteration_num, budget):
  np = numpy

  cost = 1.8

  gmm_equations = GMMEqs()

  # Uncomment the next two lines to replicate the exact runs used in the paper.
  # random_seed = 232281293 + iteration_num
  # np.random.seed(random_seed)
  df = generate_data_samples(num_samples=budget * 5, model=truth)
  np.random.seed(None)

  sample_revealer = SampleRevealer(budget=budget, cost=cost, df=df)

  log_fraction = 3 * np.log(budget) / budget
  sqrt_fraction = 1.5 / np.sqrt(budget)

  if strategy_name == "oracle":
    strategy = OracleStrategy(sample_revealer, gmm_equations=gmm_equations,
                              budget=budget, cost=cost, optimal_k=0.146906,
                              batch_fractions=[0.1, 0.9])
  elif strategy_name == "fixed_equal":
    strategy = OracleStrategy(sample_revealer, gmm_equations=gmm_equations,
                              budget=budget, cost=cost, optimal_k=0.5,
                              batch_fractions=[0.1, 0.9])
  elif strategy_name == "etc_0.2":
    strategy = ETCStrategy(sample_revealer, gmm_equations=gmm_equations,
                           budget=budget, cost=cost,
                           batch_fractions=[0.2, 0.9])
  elif strategy_name == "etc_0.4":
    strategy = ETCStrategy(sample_revealer, gmm_equations=gmm_equations,
                           budget=budget, cost=cost,
                           batch_fractions=[0.4, 0.9])
  elif strategy_name == "etg-fb_0.2":
    strategy = ETGreedyFBStrategy(sample_revealer, gmm_equations=gmm_equations,
                                  budget=budget, cost=cost,
                                  batch_fractions=[0.2, 0.4, 0.6, 0.8])
  elif strategy_name == "etg-fs_0.2":
    strategy = ETGreedyFSStrategy(sample_revealer, gmm_equations=gmm_equations,
                                  budget=budget, cost=cost,
                                  batch_fractions=[0.2, 0.4, 0.6, 0.8])
  else:
    raise ValueError("invalid strategy_name: %s" % strategy_name)

  return strategy.execute_run(cost=cost)

In [ ]:
# Test a strategy.

result = execute_strategy_iteration("etc_0.4", 1, budget=1000)
print(result.squared_errors)

[0.0013634008307765917, 0.00026331423565906334, 0.00011862935149242314]


**Executing the runs for each strategy in parallel.**

For the paper, we execute 12,000 runs for each strategy. We execute the runs
in parallel using
[`ipyparallel`](https://ipyparallel.readthedocs.io/en/latest/).

The following command starts the `ipyparallel` engines:
```
ipcluster start -n <num_engines>
```
 

In [ ]:
import ipyparallel as ipp

In [ ]:
# Verify that ipcluster is running and import the necessary Python packages.

parallel_client = ipp.Client(debug=False)
dview = parallel_client[:]
# Execute an identity map in parallel.
ar = dview.map(lambda x: x, (i for i in range(0, 2000000, 2)))
assert ar.get()[0] == 0

# Import the required Python packages.
with dview.sync_imports():
  from abc import ABC, abstractmethod
  import numpy
  import sympy
  import pandas
  import sympy
  import datetime
  import copy
  import attr
  import time
  import logging
  import itertools
  import pickle
  import os
  import functools
  import ipyparallel

  import collections

  from scipy.optimize import minimize
  import warnings

  try:
    from cPickle import dumps, loads, HIGHEST_PROTOCOL as PICKLE_PROTOCOL
  except ImportError:
    from pickle import dumps, loads, HIGHEST_PROTOCOL as PICKLE_PROTOCOL

# Make sure ipyparallel is still able to execute functions.
dview = parallel_client[:]
ar = dview.map(lambda x: x, (i for i in range(0, 2000000, 2)))
assert ar.get()[0] == 0

importing ABC,abstractmethod from abc on engine(s)
importing numpy on engine(s)
importing sympy on engine(s)
importing pandas on engine(s)
importing datetime on engine(s)
importing copy on engine(s)
importing attr on engine(s)
importing time on engine(s)
importing logging on engine(s)
importing itertools on engine(s)
importing pickle on engine(s)
importing os on engine(s)
importing functools on engine(s)
importing ipyparallel on engine(s)
importing collections on engine(s)
importing minimize from scipy.optimize on engine(s)
importing warnings on engine(s)


In [ ]:
def combine_parallel_results(async_result, need_interpolation=False):
  optimal_ks_res = []

  if not need_interpolation:
    budgets = async_result.get()[0].budgets_used
    errors = np.vstack([res.squared_errors for res in async_result.get() if res is not None])
    optimal_ks = np.vstack([[res.optimal_ks] for res in async_result.get() if res is not None])
    current_ks = np.vstack([[res.current_ks] for res in async_result.get() if res is not None])

    return budgets, errors, optimal_ks, current_ks
  
  budgets_matrix = [res.budgets_used for res in async_result.get() if res is not None]
  errors_matrix = [res.squared_errors for res in async_result.get() if res is not None]
  optimal_ks_matrix = [res.optimal_ks for res in async_result.get() if res is not None]
  current_ks_matrix = [res.current_ks for res in async_result.get() if res is not None]

  def find_budget():
    idx = 0
    target = budgets_matrix[0][-1]

    for i in range(1, len(budgets_matrix)):
      if target > budgets_matrix[i][-1]:
        target = budgets_matrix[i][-1]
        idx = i
    
    return np.array(budgets_matrix[idx])
  
  budgets = find_budget()
  errors = []
  optimal_ks = []
  current_ks = []

  for i in range(len(budgets_matrix)):
    f_err = interpolate.interp1d(budgets_matrix[i], errors_matrix[i])
    f_optimalk = interpolate.interp1d(budgets_matrix[i], np.array(optimal_ks_matrix[i])[:])
    f_currentk = interpolate.interp1d(budgets_matrix[i], np.array(current_ks_matrix[i])[:])
    
    errors.append([])
    optimal_ks.append([])
    current_ks.append([])

    for b in budgets:
      errors[i].append(f_err(b))
      optimal_ks[i].append(f_optimalk(b))
      current_ks[i].append(f_currentk(b))
  
  errors = np.array(errors)
  optimal_ks = np.array(optimal_ks)
  current_ks = np.array(current_ks)

  return budgets, errors, optimal_ks, current_ks

def execute_strategy_in_parallel(strategy_name, horizon, iterations):
  num_threads = len(parallel_client.ids)
  dview = parallel_client[:]

  print("Executing %s over %d iterations across %d cores" % (strategy_name, iterations, num_threads))

  dview["batch_fractions_to_sizes"] = batch_fractions_to_sizes
  dview["compute_reveal_k"] = compute_reveal_k
  dview["compute_reveal_k_with_budget"] = compute_reveal_k_with_budget
  dview["get_fraction_cost"] = get_fraction_cost
  dview["ModelParams"] = ModelParams
  dview["truth"] = truth
  dview["generate_data_samples"] = generate_data_samples
  dview["StrategyRunResult"] = StrategyRunResult
  dview["GMM"] = GMM
  dview["GMMEqs"] = GMMEqs
  dview["OracleStrategy"] = OracleStrategy
  dview["ETCStrategy"] = ETCStrategy
  dview["ETGreedyFBStrategy"] = ETGreedyFBStrategy
  dview["ETGreedyFSStrategy"] = ETGreedyFSStrategy
  dview["SampleRevealer"] = SampleRevealer
  dview["execute_strategy_iteration"] = execute_strategy_iteration

  def execute_iteration(i):
    return execute_strategy_iteration(strategy_name, i, horizon)

  return dview.map(execute_iteration, range(iterations))

In [ ]:
def get_timeseries_for(strategy_names, horizons, iterations, results_dict):
  results_dict["truth"] = truth

  for strategy_name in strategy_names:
    for horizon in horizons:
      print("Timestamp start: %s, Strategy: %s, Horizon: %d, Iters: %d" % (
          datetime.datetime.now(), strategy_name, horizon, iterations))
      async_result = execute_strategy_in_parallel(strategy_name, horizon, iterations)
      need_interpolation = "etg-fs" in strategy_name
      budgets, errors, optimal_ks, current_ks = combine_parallel_results(async_result, need_interpolation)
      print("Timestamp end: %s" % (datetime.datetime.now()))

      if strategy_name not in results_dict:
        results_dict[strategy_name] = {}
      
      results_dict[strategy_name][horizon] = {
          "budgets": budgets,
          "errors": errors * budgets,
          "optimal_ks": optimal_ks,
          "current_ks": current_ks,
      }

In [ ]:
results_dict = {}
get_timeseries_for([
                    "oracle", "fixed_equal",
                    "etc_0.2", "etc_0.4",
                    "etg-fb_0.2", "etg-fs_0.2",
                    ],
                   horizons=[500, 600, 700, 800, 900, 1000, 1150, 1300],
                   iterations=12*1000,
                   results_dict=results_dict)

In [ ]:
# Save results to file.
# pickle.dump(results_dict,
#             open(os.path.join(HOME_DIR,
#                               "%s_confounder_mediator.pkl" % datetime.datetime.now()),
#                  "wb"))

### Plot the results

In [ ]:
# For the color map:
# https://gist.github.com/AndiH/c957b4d769e628f506bd

# Tableau 20 Colors
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),  
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),  
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),  
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),  
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]
             
# Tableau Color Blind 10
tableau20blind = [(0, 107, 164), (255, 128, 14), (171, 171, 171), (89, 89, 89),
             (95, 158, 209), (200, 82, 0), (137, 137, 137), (163, 200, 236),
             (255, 188, 121), (207, 207, 207)]
  
# Rescale to values between 0 and 1 
for i in range(len(tableau20)):  
    r, g, b = tableau20[i]  
    tableau20[i] = (r / 255., g / 255., b / 255.)
for i in range(len(tableau20blind)):  
    r, g, b = tableau20blind[i]  
    tableau20blind[i] = (r / 255., g / 255., b / 255.)
# Use with plt.plot(…, color=tableau[0],…)

In [ ]:
def plot_regret_curve(results_dict):

  clist = rcParams['axes.prop_cycle']
  cgen = itertools.cycle(clist)

  oracle_mses = {}
  for horizon, timeseries in results_dict["oracle"].items():
    oracle_mses[timeseries["budgets"][-1]] = np.mean(timeseries["errors"][:, -1])

  oracle_budgets = np.array(list(oracle_mses.keys()))

  def plot(axs, name, info, result, with_var=False):
    x_vals = []
    y_mean = []
    y_std = []

    current_k_mean = []
    optimal_k_mean = []

    for horizon, timeseries in result.items():
      x_vals.append(timeseries["budgets"][-1])

      closest_budget = oracle_budgets[np.argmin(np.abs(oracle_budgets - x_vals[-1]))]
      y_scaled = ( timeseries["errors"][:, -1] - oracle_mses[closest_budget] )  / oracle_mses[closest_budget] * 100
      y_mean.append(np.mean(y_scaled))
      y_std.append(np.std(y_scaled))

    x_vals = np.array(x_vals)
    y_mean = np.array(y_mean)
    y_std = np.array(y_std)

    x_sort_idx = np.argsort(x_vals)
    x_vals = x_vals[x_sort_idx]
    y_mean = y_mean[x_sort_idx]
    y_std = y_std[x_sort_idx]
    
    color = next(cgen)["color"]

    plt.plot(x_vals, y_mean, label=name, color=info[1], linestyle=info[0],  marker=info[2])
    plt.legend()

    if with_var:
      ci = 1.96 * y_std / np.sqrt(timeseries["errors"].shape[0])
      axs.errorbar(x_vals, y_mean, yerr=ci, ls="none", color=info[1])
      
  name_to_linestyle_color = {
      "etc_0.2": ["dashdot", tableau20blind[1], "s"],
      "etc_0.4": ["solid", tableau20blind[2], "v"],
      "etg-fb_0.2": ["solid", tableau20blind[8], "^"],
      "etg-fs_0.2": ["dashdot", tableau20blind[6], "D"],
      "fixed_equal": ["dashed", tableau20blind[7], "o"],
  }
  plt.title("Relative regret vs budget")
  plt.xlabel("Total budget ")
  plt.ylabel("Relative regret (%)")
  for name, info in name_to_linestyle_color.items():  
    plot(plt, name, info, results_dict[name], with_var=True)
  
  # plt.savefig(os.path.join(HOME_DIR, "figures/frontdoor_backdoor_regret_curve.eps"), bbox_inches='tight', pad_inches=0.0)

In [ ]:
SMALL_SIZE = 12
MEDIUM_SIZE = 12
BIGGER_SIZE = 12

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE+4)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE+4)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE+20)  # fontsize of the figure title

In [ ]:
plot_regret_curve(results_dict)